In [2]:
from flask import Flask, request, url_for, render_template, redirect, flash
import dash
import os
import requests
import pandas as pd
import json
import plotly
import plotly.express as px
from dash import dcc, html
import numpy as np

# These come from the Player dictionary, and must be explicitly added to the loop @ **Position 1**
identifiers = ["Owner", "Week", "Player", "player_id", "age", "position"]

# These will automatically be added to the stats
default_stats = ["gp", "off_snp", "tm_off_snp", "pts_ppr", "pts_half_ppr", "pts_std"]
rush_stats = ["rush_att", "rush_yd", "rush_yac", "rush_rz_att", "rush_td"]
rec_stats = ["rec_tgt", "rec", "rec_yd", "rec_yar", "rec_rz_tgt", "rec_td"]
pass_stats = ['pass_att', 'pass_yd', 'pass_cmp', 'pass_rz_att', 'pass_2pt', 'pass_int', 'pass_sack_yds', 'pass_td']
kick_stats = ['xpm', 'xpa', 'fgm', 'fga']
def_stats = ['sacks', 'fum_rec', 'int', 'def_td', 'yds_allow', 'pts_allow', 'blk_kick']
fumble_stats = ['fum', 'fum_lost']

stat_list = default_stats + rush_stats + rec_stats + pass_stats + def_stats + fumble_stats

URL = "https://api.sleeper.app/v1/players/nfl"
response = requests.get(URL)
PLAYER_DICTIONARY = response.json()

In [9]:
def fetch_data(league_id):
    # Fetch League Metadata
    URL = f"https://api.sleeper.app/v1/league/{league_id}"
    response = requests.get(URL)
    league = response.json()

    # Fetch infomation about users in the league
    URL = f"https://api.sleeper.app/v1/league/{league_id}/users"
    response = requests.get(URL)
    users = response.json()

    # Fetch information about current rosters in the league
    URL = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
    response = requests.get(URL)
    rosters = response.json()

    # Get league name and league season
    league_name = league['name']
    season = league['season']
    playoff_week = league['settings']['playoff_week_start']

    # Create a lookup dictionary connecting owner usernames to their roster players
    owner_players = [[PLAYER_DICTIONARY[player_id] for player_id in roster['players']] for roster in rosters]
    owner_ids = [roster['owner_id'] for roster in rosters]
    ownerid2name = {owner['user_id']: owner['display_name'] for owner in users}
    owner2players = {ownerid2name[owner_ids[i]]: owner_players[i] for i in range(len(owner_ids))}
    
    # Initialize the Dataframe
    df = []
    
    # Weekly Data
    weekly_data = [requests.get(f"https://api.sleeper.app/v1/stats/nfl/regular/{season}/{week}").json()
                   for week in range(1, playoff_week+1)]
    
    # For each player in owners' rosters...
    for week, stats in enumerate(weekly_data):
        week += 1
        for owner, players in owner2players.items():
            for player in players:
                # Extract identifiers for the current player
                player_id = player['player_id']
                if 'age' in player:
                    player_age = player['age']
                else:
                    player_age = np.nan
                player_name = player['first_name'] + ' ' + player['last_name']
                player_position = player['position']
                # **Position 1**
                row = [owner, week, player_name, player_id, player_age, player_position]
                # Extract stats for the player, if they played that week
                if player_id in stats:
                    player_stats = stats[player_id]
                    # If the stat exists in player stats, extract the stat, otherwise put NaN
                    row += [player_stats[stat] if stat in player_stats else np.nan for stat in stat_list]
                # If the player did not play that week
                else:
                    # Make the entire row NaN
                    row += [np.nan] * len(stat_list)
                # Add the row to the Dataframe
                df.append(row)

    #Build the Dataframe
    columns = identifiers + stat_list
    df = pd.DataFrame(df, columns = columns)

    # Extract season-long stats
    URL = f"https://api.sleeper.app/v1/stats/nfl/regular/{season}"
    response = requests.get(URL)
    stats = response.json()

    # Initialize the Dataframe
    df_season = []

    # For each player in owners' rosters...
    for owner, players in owner2players.items():
        for player in players:
            # Extract identifying information
            player_id = player['player_id']
            if 'age' in player:
                player_age = player['age']
            else:
                player_age = np.nan
            player_name = player['first_name'] + ' ' + player['last_name']
            player_position = player['position']
            row = [owner, "Season", player_name, player_id, player_age, player_position]
            if player_id in stats:
                player_stats = stats[player_id]
                row += [player_stats[stat] if stat in player_stats else np.nan for stat in stat_list]
            else:
                row += [np.nan] * len(stat_list)
            df_season.append(row)

    # Build the Dataframe
    df_season = pd.DataFrame(df_season, columns = columns)

    stats = pd.concat([df, df_season])

    return stats

In [10]:
from datetime import datetime

league_id = 716027744466583552

start = datetime.now()
fetch_data(league_id)
print(datetime.now() - start)

0:00:09.727531
